## Step 1：初始化Overlay以下载bitstream

In [ ]:
from pynq import Overlay
from pynq.lib.iic import *
from pynq.lib.video2lcd import *
from pynq.lib.ov5640 import OV5640

ov5640_ol = Overlay("ov5640_lcd_7020.bit")

## Step 2：实例化I2C以配置OV5640 Camera

In [ ]:
iic = AxiIIC(ov5640_ol.ip_dict['cam_iic'])
# OV5640器件地址
address = 0x3c  
ov5640= OV5640(address, iic)

### 初始化OV5640

In [ ]:
ov5640.init()

### 初始化OV5640自动聚焦功能

In [ ]:
ov5640.focus_init()

### 使能自动聚焦功能

In [ ]:
ov5640.focus_continue()

## Step 3：根据LCD的ID配置OV5640的输出分辨率

### 实例化视频输入video_in与视频输出video_out

In [ ]:
video_in = ov5640_ol.video.video_in
video_out = ov5640_ol.video.video_out

### 读取lcd id并选择相应分辨率参数

In [ ]:
lcd_id = video_out.lcd_id_rd()
print("lcd_id=",hex(lcd_id))
# 4.3寸480*272 RGB屏,ID=0X4342
# 7寸800*480 RGB屏,ID=0X7084
# 7寸1024*600 RGB屏,ID=0X7016
# 4.3寸800*480 RGB屏,ID=0X4384
# 10.1寸1280*800 RGB屏,ID=0X1018
# 其他值,非法ID;

#根据获取的LCD的ID号来进行ov5640显示分辨率参数的选择
if lcd_id == 0x4342 :  #4.3寸屏,480*272分辨率
    cmos_h_pixel  = 480 
    cmos_v_pixel  = 272 
    total_h_pixel = 1800 
    total_v_pixel = 1000 
elif lcd_id == 0x4384 :  #4.3寸屏,800*480分辨率
    cmos_h_pixel  = 800 
    cmos_v_pixel  = 480 
    total_h_pixel = 1800 
    total_v_pixel = 1000 
elif lcd_id == 0x7084 :  #7寸屏,800*480分辨率
    cmos_h_pixel  = 800 
    cmos_v_pixel  = 480 
    total_h_pixel = 1800 
    total_v_pixel = 1000 
elif lcd_id == 0x7016 :  #7寸屏,1024*600分辨率
    cmos_h_pixel  = 1024 
    cmos_v_pixel  = 600 
    total_h_pixel = 2200 
    total_v_pixel = 1000 
elif lcd_id == 0x1018 :  #10.1寸屏,1280*800分辨率
    cmos_h_pixel  = 1280
    cmos_v_pixel  = 800
    total_h_pixel = 2570
    total_v_pixel = 980
else:
    cmos_h_pixel  = 480
    cmos_v_pixel  = 272
    total_h_pixel = 1800
    total_v_pixel = 1000

### 配置OV5640的输出分辨率

In [ ]:
ov5640.set_outsize(cmos_h_pixel,cmos_v_pixel,total_h_pixel,total_v_pixel)

## Step 4: 配置video_in与video_out并驱动显示

In [ ]:
# mode = VideoMode(800, 480, 24)
video_out.configure(1)
video_in.configure(video_out.mode)
video_in.start()
video_out.start()

video_in.tie(video_out)

## Capture and display one frame of picture, it may takes 3~5 seconds

In [ ]:
while True:
    frame = video_in.readframe()
    video_out.writeframe(frame)
    if(frame.max() != 0):
        break;

In [ ]:
from PIL import Image
import IPython
image = Image.fromarray(frame)
g, b, r = image.split()
image = Image.merge('RGB', [r,b,g])
IPython.display.clear_output(1)
display(image)

## 结束
关闭Video输入与输出

In [ ]:
video_in.close()
video_out.close()